In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Generate sample dataset for training
np.random.seed(42)
data = pd.DataFrame({
    'Salary': np.random.randint(2000, 10000, 200),
    'Age': np.random.randint(18, 65, 200),
    'Credit Score': np.random.randint(300, 850, 200),
    'Academic Score': np.random.randint(50, 100, 200),
    'Eligible': np.random.choice([0, 1], 200)  # 1: Eligible, 0: Not Eligible
})

X = data[['Salary', 'Age', 'Credit Score', 'Academic Score']]
y = data['Eligible']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

def loan_eligibility_check(salary, age, credit_score, academic_score):
    input_data = np.array([[salary, age, credit_score, academic_score]])
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    return "Eligible for Loan" if prediction == 1 else "Not Eligible"

# OCR & Visualization function (placeholder)
def ocr_and_visualize(uploaded_images, chart_type, document_type):
    # Placeholder extracted data
    extracted_data = pd.DataFrame({
        "Field": ["Revenue", "Expenses", "Profit"],
        "Value": [50000, 30000, 20000]
    })

    # Generate a dummy visualization chart
    chart_path = "chart.png"
    extracted_data.plot(kind="bar" if chart_type == "Bar Chart" else "pie", y="Value", x="Field")
    plt.savefig(chart_path)

    return extracted_data.to_html(), chart_path

def create_ui():
    with gr.Blocks() as interface:
        gr.Markdown("# 📄 OCR & Loan Eligibility Checker 📄")

        with gr.Tabs():
            with gr.Tab("OCR & Data Visualization"):
                gr.Markdown("### Upload an Image for OCR & Data Visualization")
                with gr.Row():
                    uploaded_images = gr.File(label="Upload Image", file_types=[".png", ".jpg", ".jpeg"])
                    chart_type = gr.Dropdown(choices=["Bar Chart", "Pie Chart"], value="Bar Chart", label="Chart Type")
                    document_type = gr.Dropdown(choices=["Balance Sheet", "Profit & Loss", "Payslip"], value="Balance Sheet", label="Document Type")
                submit_btn = gr.Button("Extract & Visualize")
                data_table = gr.HTML(label="Extracted Data Table")
                chart = gr.Image(label="Visualization Chart", type="filepath")
                submit_btn.click(ocr_and_visualize, inputs=[uploaded_images, chart_type, document_type], outputs=[data_table, chart])

            with gr.Tab("Loan Eligibility"):
                gr.Markdown("### Loan Eligibility Check")
                with gr.Row():
                    salary = gr.Number(label="Monthly Salary")
                    age = gr.Number(label="Age")
                    credit_score = gr.Number(label="Credit Score")
                    academic_score = gr.Number(label="Academic Score")
                check_btn = gr.Button("Check Eligibility")
                result = gr.Textbox(label="Eligibility Result")
                check_btn.click(loan_eligibility_check, inputs=[salary, age, credit_score, academic_score], outputs=result)

    return interface

if __name__ == "__main__":
    create_ui().launch(debug=True)

In [ ]:
import gradio as gr
import pandas as pd
import pickle

# Load pre-trained model (if available)
try:
    model = pickle.load(open('loan_recommendation_model.pkl', 'rb'))
except:
    model = None  # Placeholder if the model isn't trained yet

# Sample Loan Data
loans = pd.DataFrame({
    'Loan Type': ['Education Loan A', 'Education Loan B', 'Education Loan C'],
    'Interest Rate': [7.5, 8.0, 9.2],
    'Max Amount': [500000, 1000000, 2000000],
    'Eligibility Score': [600, 650, 700]
})

def recommend_loans(score, amount_needed):
    """Recommend loans based on eligibility score and amount needed."""
    eligible_loans = loans[(loans['Eligibility Score'] <= score) & (loans['Max Amount'] >= amount_needed)]
    return eligible_loans if not eligible_loans.empty else "No suitable loans found."

def find_loans(score, amount_needed):
    results = recommend_loans(score, amount_needed)
    return results.to_string(index=False) if isinstance(results, pd.DataFrame) else results

def check_status(app_id):
    # Placeholder tracking data
    tracking_data = pd.DataFrame({
        'Application ID': [101, 102, 103],
        'Status': ['Approved', 'Pending', 'Rejected']
    })
    status = tracking_data[tracking_data['Application ID'] == int(app_id)]
    return status.iloc[0]['Status'] if not status.empty else "Application ID not found."

def submit_report(report):
    return "Your report has been submitted successfully."

# Gradio Interface
loan_interface = gr.Interface(
    fn=find_loans,
    inputs=[
        gr.Slider(minimum=300, maximum=850, value=650, label="Enter your credit/eligibility score:"),
        gr.Number(value=50000, label="Enter loan amount needed (in INR):")
    ],
    outputs="text",
    title="Loan Advisory System",
    description="This system helps students find suitable education loans."
)

status_interface = gr.Interface(
    fn=check_status,
    inputs=gr.Number(label="Enter Application ID:"),
    outputs="text",
    title="Loan Application Status Tracking"
)

deployment_interface = gr.Interface(
    fn=submit_report,
    inputs=gr.Textbox(label="Enter your daily deployment update:"),
    outputs="text",
    title="Deployment Progress Updates"
)

# Launch the Gradio App
gr.TabbedInterface(
    [loan_interface, status_interface, deployment_interface],
    ["Find Loans", "Check Status", "Deployment Updates"]
).launch()

In [ ]:
import gradio as gr
import pandas as pd
import pickle
import pytesseract
import matplotlib.pyplot as plt
from PIL import Image

# Load pre-trained model (if available)
try:
    model = pickle.load(open('loan_recommendation_model.pkl', 'rb'))
except:
    model = None  # Placeholder if the model isn't trained yet

# Sample Loan Data
loans = pd.DataFrame({
    'Loan Type': ['Education Loan A', 'Education Loan B', 'Education Loan C'],
    'Interest Rate': [7.5, 8.0, 9.2],
    'Max Amount': [500000, 1000000, 2000000],
    'Eligibility Score': [600, 650, 700]
})

def recommend_loans(score, amount_needed):
    """Recommend loans based on eligibility score and amount needed."""
    eligible_loans = loans[(loans['Eligibility Score'] <= score) & (loans['Max Amount'] >= amount_needed)]
    return eligible_loans if not eligible_loans.empty else "No suitable loans found."

def find_loans(score, amount_needed):
    results = recommend_loans(score, amount_needed)
    return results.to_string(index=False) if isinstance(results, pd.DataFrame) else results

def check_status(app_id):
    # Placeholder tracking data
    tracking_data = pd.DataFrame({
        'Application ID': [101, 102, 103],
        'Status': ['Approved', 'Pending', 'Rejected']
    })
    status = tracking_data[tracking_data['Application ID'] == int(app_id)]
    return status.iloc[0]['Status'] if not status.empty else "Application ID not found."

def submit_report(report):
    return "Your report has been submitted successfully."

def extract_text_from_image(image):
    """Extract text from an uploaded image and convert it into a CSV file."""
    img = Image.open(image)
    extracted_text = pytesseract.image_to_string(img)
    
    # Convert text into a DataFrame
    data = {'Extracted Text': [extracted_text]}
    df = pd.DataFrame(data)
    csv_filename = "extracted_text.csv"
    df.to_csv(csv_filename, index=False)
    
    # Visualization (Bar Chart and Pie Chart)
    word_counts = pd.Series(extracted_text.split()).value_counts()[:10]  # Top 10 words
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Bar Chart
    word_counts.plot(kind='bar', ax=axes[0], title='Top 10 Word Frequency')
    
    # Pie Chart
    word_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', title='Word Distribution')
    
    plt.tight_layout()
    
    return csv_filename, fig

# Gradio Interfaces
loan_interface = gr.Interface(
    fn=find_loans,
    inputs=[
        gr.Slider(minimum=300, maximum=850, value=650, label="Enter your credit/eligibility score:"),
        gr.Number(value=50000, label="Enter loan amount needed (in INR):")
    ],
    outputs="text",
    title="Loan Eligibility Check",
    description="This system helps students find suitable education loans."
)

status_interface = gr.Interface(
    fn=check_status,
    inputs=gr.Number(label="Enter Application ID:"),
    outputs="text",
    title="Loan Application Status Tracking"
)

deployment_interface = gr.Interface(
    fn=submit_report,
    inputs=gr.Textbox(label="Enter your daily deployment update:"),
    outputs="text",
    title="Deployment Progress Updates"
)

document_analysis_interface = gr.Interface(
    fn=extract_text_from_image,
    inputs=gr.Image(type="filepath", label="Upload Document Image"),
    outputs=["file", "plot"],
    title="Document Analysis",
    description="Upload an image of a document to extract text, convert it to CSV, and visualize word frequency."
)

# Launch the Gradio App
gr.TabbedInterface(
    [loan_interface, status_interface, deployment_interface, document_analysis_interface],
    ["Loan Eligibility", "Check Status", "Deployment Updates", "Document Analysis"]
).launch()


In [2]:
import gradio as gr
import pandas as pd
import pickle
import pytesseract
import matplotlib.pyplot as plt
from PIL import Image

# Load pre-trained model (if available)
try:
    model = pickle.load(open('loan_recommendation_model.pkl', 'rb'))
except:
    model = None  # Placeholder if the model isn't trained yet

# Sample Loan Data
loans = pd.DataFrame({
    'Loan Type': ['Education Loan A', 'Education Loan B', 'Education Loan C'],
    'Interest Rate': [7.5, 8.0, 9.2],
    'Max Amount': [500000, 1000000, 2000000],
    'Eligibility Score': [600, 650, 700],
    'Min Age': [18, 20, 21],
    'Min Academic Score': [60, 65, 70]
})

def recommend_loans(score, age, academic_score, amount_needed):
    """Check if the user is eligible for any loans based on criteria."""
    eligible_loans = loans[
        (loans['Eligibility Score'] <= score) &
        (loans['Max Amount'] >= amount_needed) &
        (loans['Min Age'] <= age) &
        (loans['Min Academic Score'] <= academic_score)
    ]
    return "Eligible" if not eligible_loans.empty else "Not Eligible"

def find_loans(score, age, academic_score, amount_needed):
    return recommend_loans(score, age, academic_score, amount_needed)

def extract_text_from_image(image):
    """Extract text from an uploaded image and convert it into a CSV file."""
    img = Image.open(image)
    extracted_text = pytesseract.image_to_string(img)
    
    # Convert text into a DataFrame
    data = {'Extracted Text': [extracted_text]}
    df = pd.DataFrame(data)
    csv_filename = "extracted_text.csv"
    df.to_csv(csv_filename, index=False)
    
    # Visualization (Bar Chart and Pie Chart)
    word_counts = pd.Series(extracted_text.split()).value_counts()[:10]  # Top 10 words
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Bar Chart
    word_counts.plot(kind='bar', ax=axes[0], title='Top 10 Word Frequency')
    
    # Pie Chart
    word_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', title='Word Distribution')
    
    plt.tight_layout()
    
    return csv_filename, fig

# Gradio Interfaces
loan_interface = gr.Interface(
    fn=find_loans,
    inputs=[
        gr.Slider(minimum=300, maximum=850, value=650, label="Enter your credit/eligibility score:"),
        gr.Number(value=18, label="Enter your age:"),
        gr.Number(value=60, label="Enter your academic score (percentage):"),
        gr.Number(value=50000, label="Enter loan amount needed (in INR):")
    ],
    outputs="text",
    title="Loan Eligibility Check",
    description="This system helps students check if they are eligible for education loans based on credit score, age, and academic performance."
)

document_analysis_interface = gr.Interface(
    fn=extract_text_from_image,
    inputs=gr.Image(type="filepath", label="Upload Document Image"),
    outputs=["file", "plot"],
    title="Document Analysis",
    description="Upload an image of a document to extract text, convert it to CSV, and visualize word frequency."
)

# Launch the Gradio App
gr.TabbedInterface(
    [loan_interface, document_analysis_interface],
    ["Loan Eligibility", "Document Analysis"]
).launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [9]:
import gradio as gr
import pandas as pd
import time
from pymongo import MongoClient
import plotly.express as px

# MongoDB Connection
MONGO_URI = "mongodb://localhost:27017/"
DB_SUPERVISED = "supervised"
DB_UNSUPERVISED = "unsupervised"
DB_SEMI_SUPERVISED = "semi_supervised"

client = MongoClient(MONGO_URI)
db_supervised = client[DB_SUPERVISED]
db_unsupervised = client[DB_UNSUPERVISED]
db_semi_supervised = client[DB_SEMI_SUPERVISED]

def display_insights(categories):
    if categories:
        return f"\U0001F4A1 You've spent a large amount on {', '.join(categories)}. Consider reviewing your spending habits."
    return "\U0001F4A1 Your spending seems balanced across categories."

def stock_insights(stock_symbol):
    insights = {
        "AAPL": "\U0001F4C9 Apple’s stock has been on a rollercoaster lately. Keep an eye out for any major news events!",
        "GOOGL": "\U0001F4BC Google's stock is still holding strong, but you might want to think about diversifying your investments.",
        "MSFT": "\U0001F4C8 Microsoft has been outperforming! But remember, the market can change quickly. Stay alert!"
    }
    return insights.get(stock_symbol, "\U0001F4CA Keep monitoring the market closely!")

def process_files(doc_type, stock_choice1=None):
    time.sleep(3)  # Simulating backend processing
    if doc_type == "Invoice":
        collection = db_supervised["invoice_classified"]
        data = pd.DataFrame(list(collection.find({}, {"_id": 0})))
        fig = px.pie(data, names='category', values='price', title="Invoice Category Distribution")
        insights = display_insights(data['category'].unique())
        return data, fig, insights

    elif doc_type == "Payslip":
        collection = db_supervised["payslip_classified"]
        data = pd.DataFrame(list(collection.find({}, {"_id": 0})))
        fig = px.pie(data, names='Description', values='Price', title="Payslip Description Distribution")
        insights = display_insights(data['Description'].unique())
        return data, fig, insights

    elif doc_type == "Stock Market" and stock_choice1:
        collection = db_semi_supervised["api"]
        stock_data = collection.find_one({"symbol": stock_choice1})
        if stock_data:
            stock_dates = list(stock_data["data"].keys())
            stock_close = [float(stock_data["data"][date]["4. close"]) for date in stock_dates]
            df_stock = pd.DataFrame({'Date': pd.to_datetime(stock_dates), 'Close Price': stock_close})
            fig = px.line(df_stock, x='Date', y='Close Price', title=f"{stock_choice1} Stock Price Trend")
            insights = stock_insights(stock_choice1)
            return df_stock, fig, insights
    
    return "No data found", None, "No insights available"

iface = gr.Interface(
    fn=process_files,
    inputs=[
        gr.Radio(["Invoice", "Payslip", "Stock Market"], label="Select Document Type"),
        gr.Dropdown(["AAPL", "GOOGL", "MSFT"], label="Select Stock Symbol (if applicable)", allow_none=True)
    ],
    outputs=[gr.Dataframe(), gr.Plot(), gr.Text()],
    title="Finsight - Financial Document Classifier",
    description="Upload financial documents and get insights."
)

iface.launch()


TypeError: Dropdown.__init__() got an unexpected keyword argument 'allow_none'

In [ ]:
from flask import Flask, request, jsonify
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import os
import google.generativeai as genai
from dotenv import load_dotenv
from flask_cors import CORS  # Enable CORS for frontend communication

# Load API Key securely
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Load BLIP model for image captioning
model_name = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)

app = Flask(__name__)
CORS(app)  # Enable CORS for frontend communication

def generate_caption(image):
    """Generate a caption using BLIP."""
    inputs = processor(images=image, return_tensors="pt")
    caption_ids = model.generate(**inputs)
    return processor.batch_decode(caption_ids, skip_special_tokens=True)[0]

def generate_detailed_description(caption):
    """Generate a detailed description using Google Gemini AI."""
    model = genai.GenerativeModel("gemini-1.5-flash")
    prompt = f"Provide a detailed description of this image caption: '{caption}'"
    
    response = model.generate_content(prompt)
    return response.text

@app.route('/generate_caption', methods=['POST'])
def upload_image():
    if "image" not in request.files:
        return jsonify({"error": "No image uploaded"}), 400

    file = request.files["image"]
    image = Image.open(file.stream).convert("RGB")

    caption = generate_caption(image)
    description = generate_detailed_description(caption)

    return jsonify({"caption": caption, "description": description})

if __name__ == "__main__":
    app.run(debug=True)


c:\Users\yguru\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yguru\.cache\huggingface\hub\models--Salesforce--blip-image-captioning-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Error while downloading from https://cdn-lfs.hf.co/repos/f1/cb/f1cbe4cfb2a26